# Setup Preamble



In [ ]:
import os
import logging

# Scientific libraries
import pandas as pd
from Bio import SeqIO

# Graphic libraries
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# other imports
import ete3
from FastOMA import _utils_roothog
from FastOMA.zoo.hog import extract_hog_info
_utils_roothog.logger_hog.setLevel(logging.INFO)

In [ ]:
# Extra options
pd.set_option('max_colwidth',200)
pd.options.display.max_rows = 150

#style options
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12.0, 8.0)

#seaborn options
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'}, font_scale=1.5)
sns.set_style('whitegrid')
palette="husl"

In [ ]:
output_folder = "Output"
input_folder = "testdata/in_folder"
proteome_folder = input_folder + "/proteome"

# Stats on input dataset

## Proteomes 

We first show some statistics on the input proteomes, e.g. size distributions, length distributions etc, which can indicate problems in annotations and raise further problems in downstream analyses.

In [ ]:
def get_protein_dataframe(folder):
    species, seq_reqs, _ = _utils_roothog.parse_proteomes(folder)
    print(f"There are {len(species)} species in the proteome folder.")
    print(f"In total, {sum(len(z) for z in seq_reqs.values())} proteins are in the dataset.")
    print("\nThe list of species is the following:")
    prot_df = []
    for sp in species:
        print(f" - {sp} (with {len(seq_reqs[sp])} proteins)")
        sp_df = pd.DataFrame({"prot_len": [len(z) for z in seq_reqs[sp]], "species": [sp for _ in range(len(seq_reqs[sp]))]})
        prot_df.append(sp_df)
    return pd.concat(prot_df)

protein_df = get_protein_dataframe(proteome_folder)

Next follows a visual overview of the size distribution of all these proteomes.

In [ ]:
sns.histplot(data=protein_df.groupby("species", as_index=False).count(), stat="probability", bins=30)
plt.xlabel("Nr proteins in each species")
plt.ylabel("Frequency")
plt.title("Distribution of number of proteins per species")

In [ ]:
ax = sns.countplot(x="species", data=protein_df, 
                   order = protein_df['species'].value_counts().index,
                   hue = "species")
plt.xticks(rotation=90)
plt.title("Number of proteins per genome", fontsize=20)

Furthermore, a high proportion of proteins short in length may indicated a problem with genome quality. Below, we report the protein length distribution of all species used in this run. Suspect genomes are outlier peaks with a high number of short proteins.

In [ ]:
sns.displot(protein_df, x="prot_len", hue="species", kind="kde", height=8)
plt.xlim(0, 2000)
plt.title("Protein length distribution per species", fontsize=20)


## Species tree 

Next, we look at the species input species tree. Note that errors in it can lead to wrong ortholog / paralog inference. FastOMA has identified on every internal level the set of orthologous groups in a hierarchically nested way, the Hierarchcial Orthologous Groups (HOGs).
With tools such as pyHAM you can extract the group memebers in a programmable way for any internal level of the below species tree.

In [ ]:
species_tree = ete3.Tree(os.path.join(output_folder, "species_tree_checked.nwk"),format=1)
num_leaves = 0
num_internal = 0
for node in species_tree.traverse(strategy="postorder"):
    if node.is_leaf():
        num_leaves += 1
    else:
        num_internal += 1
print(f"There are {num_internal} taxonomic levels in the input species tree with {num_leaves} species as leaves.")

print(species_tree.get_ascii(show_internal=True))

# Stats on Orthoxml

In [ ]:
hog_file = os.path.join(output_folder, "FastOMA_HOGs.orthoxml")
genome_coverage_stats = extract_hog_info.SpeciesAnalyser()
with open(hog_file, 'rt') as xml:
    hog_df = pd.DataFrame.from_records(extract_hog_info.parse_orthoxml(xml, genome_coverage_stats))
hog_summary_df = pd.DataFrame.from_records(genome_coverage_stats.summary())
df_seq = pd.merge(hog_summary_df, protein_df.groupby("species", as_index=False).count(), on='species')
df_seq['minor_splice'] = df_seq['prot_len']-df_seq['genes']
df_seq = df_seq[['species', 'genes', 'not_in_group','minor_splice']]
df_seq

## Genes in HOGs

First, let's check the fraction of genes that are in any HOG per species. Note that OMA will only use one isoform per gene (if properly annotated). The toal barchart height will indicated the total number of proteins in the fasta input file. The blue part (genes) is the number of genes that are in a HOG. 

In [ ]:
df_seq.set_index('species').plot(kind='bar', stacked=True)
plt.title('Number of proteins in HOGs / singltons / minor splice variants', fontsize=16)
plt.xlabel('Species')
plt.ylabel('Counts')
plt.xticks(rotation=45);

## Group sizes

As HOGs are nested groups, we first look at the rootlevel HOGs, so the deepest level of each HOG. The level itself can be at any of the internal taxonomic levels

In [ ]:
hog_df

In [ ]:
sns.scatterplot(data=hog_df[hog_df['is_roothog']==True], x='nr_members', y='CompletenessScore')

In [ ]:
sns.jointplot(data=hog_df[hog_df['is_roothog']==True], x='nr_members', y='CompletenessScore')

# Get duplications/losses/gains/retained with PyHAM

In [ ]:
import pyham       
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s %(name)-12s %(levelname)-8s %(message)s")

In [ ]:
nwk_path= os.path.join(output_folder, "species_tree_checked.nwk") # species tree should be pruned (no extra leaves)
orthoxml_path=os.path.join(output_folder, "FastOMA_HOGs.orthoxml")
ham_analysis = pyham.Ham(nwk_path, orthoxml_path, tree_format="newick", use_internal_name=True)
print("Ham analysis done") # for a big orthoxml file it can take ~30mins

## Generate Phylostratigraphy plots

create tree profile, classify all genomes by extant or ancestral, and get % of dup, lost, retained, and gained

In [ ]:
phylostratigraphy = os.path.join(output_folder, "phylostratigraphy.html")
treeprofile = ham_analysis.create_tree_profile(outfile=phylostratigraphy)

from IPython.display import IFrame
IFrame(os.path.basename(phylostratigraphy), width=800, height=600)